In [1]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [184]:
data_good = ["I like machine learning",
        "I love coding in python",
        "I love building chatbots",
        "I don't hate you",
        "they chat amagingly well"]

tagged_data_good = [TaggedDocument(words=word_tokenize(_d.lower()), tags=['GOOD']) for i, _d in enumerate(data_good)]

data_bad = ["I don't like machine learning",
        "I hate coding in python",
        "I really hate building chatbots",
        "they chat really bad"]

tagged_data_bad = [TaggedDocument(words=word_tokenize(_d.lower()), tags=['BAD']) for i, _d in enumerate(data_bad)]

In [185]:
from sklearn import utils
from tqdm import tqdm
import multiprocessing
cores = multiprocessing.cpu_count()

tagged_data = tagged_data_good+ tagged_data_bad

In [195]:
model_dbow = Doc2Vec(dm=1, 
                     vector_size=300, 
                     negative=5, 
                     hs=0, 
                     min_count=2, 
                     sample = 0, 
                     workers=cores)

model_dbow.build_vocab([x for x in tqdm(tagged_data)])

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(tagged_data)]), total_examples=len(tagged_data), epochs=10)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha


model.save("d2v.model")
print("Model Saved")

100%|██████████| 9/9 [00:00<00:00, 25802.28it/s]


Model Saved


In [196]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("I don't hate".lower())

print('test data', test_data)

# model.most_similar('learning')

v = model.infer_vector(test_data)
model.docvecs.most_similar([v], topn=len(model.docvecs))


test data ['i', 'do', "n't", 'hate']


[('BAD', 0.9999945759773254), ('GOOD', 0.999988853931427)]

In [199]:
model.most_similar('hate')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('love', 0.999557375907898),
 ('coding', 0.9995526075363159),
 ('machine', 0.9995464086532593),
 ('learning', 0.999544084072113),
 ('chat', 0.9995251893997192),
 ('like', 0.9995152950286865),
 ('python', 0.9995043873786926),
 ('well', 0.999496340751648),
 ('amagingly', 0.9994792938232422),
 ('bad', 0.9994626045227051)]

In [200]:
def vec_for_learning(model, tagged_docs):
        sents = tagged_docs
        targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
        return targets, regressors

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)